### hadoop ecosystem

1. hadoop

2. hive

### install hadoop

\$ brew install hadoop

/usr/local/Cellar/hadoop/2.7.0

### config hadoop-env.sh

\$ pico libexec/etc/hadoop/hadoop-env.sh

replace export HADOOP_OPTS="$HADOOP_OPTS -Djava.net.preferIPv4Stack=true"

with export HADOOP_OPTS="$HADOOP_OPTS -Djava.net.preferIPv4Stack=true -Djava.security.krb5.realm= -Djava.security.krb5.kdc="

### config mapred-site.xml

\$ pico libexec/etc/hadoop/mapred-site.xml

<configuration>
    <property>
        <name>mapred.job.tracker</name>
        <value>localhost:9010</value>
    </property>
</configuration>

### config hdfs-site.xml

\$ pico libexec/etc/hadoop/hdfs-site.xml

<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration> 

### config HADOOP_HOME and alias 

\$ pico ~/.bash_profile

export HADOOP_HOME=/usr/local/Cellar/hadoop/2.7.0

alias hstart="/usr/local/Cellar/hadoop/2.7.0/sbin/start-dfs.sh;/usr/local/Cellar/hadoop/2.7.0/sbin/start-yarn.sh"
alias hstop="/usr/local/Cellar/hadoop/2.7.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.7.0/sbin/stop-dfs.sh"

\$ source ~/.bash_profile

### config ssh

\$ ssh-keygen -t ras

enable remote login

    system preferences -> sharing -> remote login

authorize ssh keys

    \$ cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
    
login via ssh

    \$ ssh localhost

### format start stop

\$ hdfs namenode -format

\$ hstart

\$ jps

\$ hadoop jar path/to/hadoop-examples.jar pi 10 100

\$ hdfs dfs -mkdir -p /user/hqlgree2

\$ hdfs dfs -put book.txt /user/hqlgree2/

\$ hstop